In [ ]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
import string
from sklearn import linear_model
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [ ]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [ ]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [ ]:
answers = {}

In [ ]:
# Some data structures that will be useful

In [ ]:
allRatings = []
for l in readCSV("drive/MyDrive/CSE 258/train_Interactions.csv.gz"):
  allRatings.append(l)

In [ ]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
UsersPerItem = defaultdict(set)
ItemsPerUser = defaultdict(set)
for u,b,r in ratingsTrain:
  ratingsPerUser[u].append((b,r))
  ratingsPerItem[b].append((u,r))
  UsersPerItem[b].add(u)
  ItemsPerUser[u].add(b)

TypeError: cannot unpack non-iterable int object

In [ ]:
##################################################
# Read prediction                                #
##################################################

In [ ]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("drive/MyDrive/CSE 258/train_Interactions.csv.gz"):
  bookCount[book] += 1
  totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalRead/2: break

In [ ]:
### Question 1

In [ ]:
y = [True] * len(ratingsValid)
allItems = set([book for _,book,_ in allRatings])
visited = set()
for user,_,_ in ratingsValid:
  if user not in visited:
    visited.add(user)
    unread_list = allItems - ItemsPerUser[user]
    book = random.choice(list(unread_list))
    ratingsValid.append((user, book, -1))
    y.append(False)

In [ ]:
from sklearn.metrics import accuracy_score
def accuracy(predictions, y):
  accuracy = accuracy_score(y, predictions)
  return accuracy

In [ ]:
pred1 = []
for _,book,_ in ratingsValid:
  pred1.append(book in return1)
acc1 = accuracy(pred1, y)

In [ ]:
answers['Q1'] = acc1

In [ ]:
assertFloat(answers['Q1'])

In [ ]:
### Question 2

In [ ]:
threshold = 0.75

In [ ]:
return2 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return2.add(i)
  if count > totalRead*threshold: break

In [ ]:
pred2 = []
for _,book,_ in ratingsValid:
  pred2.append(book in return2)
acc2 = accuracy(pred2, y)

In [ ]:
answers['Q2'] = [threshold, acc2]

In [ ]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

In [ ]:
### Question 3/4

In [ ]:
def Jaccard(s1, s2):
  intersection = len(s1.intersection(s2))
  union = len(s1.union(s2))
  jaccard_similarity = intersection / union
  return jaccard_similarity

In [ ]:
def r(u,b,threshold):
  if len(ratingsPerUser[u]) == 0:
    return False
  js_max = 0
  for b_ in ItemsPerUser[u]:
    if b_ != b:
      js = Jaccard(UsersPerItem[b], UsersPerItem[b_])
      if js > js_max:
        js_max = js
  return js_max > threshold

In [ ]:
pred3 = []
for user,book,_ in ratingsValid:
  pred3.append(r(user,book,0.003))
acc3 = accuracy(pred3, y)

In [ ]:
pred4 = []
for i in range(len(ratingsValid)):
  pred4.append(pred3[i] or pred2[i])
acc4 = accuracy(pred4, y)

In [ ]:
answers['Q3'] = acc3
answers['Q4'] = acc4

In [ ]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [ ]:
predictions = open("predictions_Read.csv", 'w')
for l in open("drive/MyDrive/CSE 258/pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    # (etc.)

predictions.close()

In [ ]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [ ]:
assert type(answers['Q5']) == str

In [ ]:
##################################################
# Rating prediction                              #
##################################################

In [ ]:
### Question 6

In [ ]:
ratingsValid = ratingsValid[:10000]
labels = [r for _,_,r in ratingsValid]
mu = sum([r for _,_,r in ratingsTrain]) / len(ratingsTrain)

In [ ]:
userIDs = {}
itemIDs = {}
for u,i,r in allRatings:
  if not u in userIDs: userIDs[u] = len(userIDs)
  if not i in itemIDs: itemIDs[i] = len(itemIDs)

In [ ]:
class LatentFactorModelBiasOnly(tf.keras.Model):
  def __init__(self, mu, lamb):
    super(LatentFactorModelBiasOnly, self).__init__()
    self.alpha = self.add_weight(name='alpha', shape=(), initializer=tf.initializers.constant(mu), trainable=True)
    self.betaU = self.add_weight(name='betaU', shape=[len(userIDs)], initializer=tf.random_normal_initializer(stddev=0.001), trainable=True)
    self.betaI = self.add_weight(name='betaI', shape=[len(itemIDs)], initializer=tf.random_normal_initializer(stddev=0.001), trainable=True)
    self.lamb = lamb

  def predict(self, u, i):
    return self.alpha + self.betaU[u] + self.betaI[i]

  def call(self, sampleU, sampleI, sampleR):
    u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
    i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
    pred = self.alpha +\
        tf.nn.embedding_lookup(self.betaU, u) +\
        tf.nn.embedding_lookup(self.betaI, i)
    r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
    return tf.nn.l2_loss(pred - r) / len(sampleR) +\
        self.lamb * (tf.reduce_sum(self.betaU**2) + tf.reduce_sum(self.betaI**2))

In [ ]:
def trainingStepBiasOnly(model, ratingsTrain, optimizer):
  sampleU, sampleI, sampleR = [], [], []
  for u, i, r in ratingsTrain:
    sampleU.append(userIDs[u])
    sampleI.append(itemIDs[i])
    sampleR.append(r)

  with tf.GradientTape() as tape:
    loss = model(sampleU=sampleU, sampleI=sampleI, sampleR=sampleR)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for (grad, var) in zip(gradients, model.trainable_variables) if grad is not None)

  return loss.numpy()

In [ ]:
def MSE(predictions, labels):
  differences = [(x-y)**2 for x,y in zip(predictions,labels)]
  return sum(differences) / len(differences)

In [ ]:
optimizer6 = tf.keras.optimizers.Adam(0.1)
lamb6 = 1

modelBiasOnly = LatentFactorModelBiasOnly(mu, lamb6)
for i in range(30):
  obj = trainingStepBiasOnly(modelBiasOnly, ratingsTrain, optimizer6)

KeyboardInterrupt: 

In [ ]:
predictions = [modelBiasOnly.predict(userIDs[u],itemIDs[i]).numpy() for u,i,_ in ratingsValid]
validMSE = MSE(predictions, labels)

In [ ]:
answers['Q6'] = validMSE

In [ ]:
assertFloat(answers['Q6'])

In [ ]:
### Question 7

In [ ]:
bu = modelBiasOnly.betaU.numpy().tolist()
maxBeta = max(bu)
minBeta = min(bu)
maxUserIndex = bu.index(maxBeta)
minUserIndex = bu.index(minBeta)
maxUser = [key for key, value in userIDs.items() if value == maxUserIndex][0]
minUser = [key for key, value in userIDs.items() if value == minUserIndex][0]

In [ ]:
answers['Q7'] = [maxUser, minUser, maxBeta, minBeta]

In [ ]:
assert [type(x) for x in answers['Q7']] == [str, str, float, float]

In [ ]:
### Question 8

In [ ]:
optimizer8 = tf.keras.optimizers.Adam(0.1)
lamb8 = 0.01

modelBiasOnly8 = LatentFactorModelBiasOnly(mu, lamb8)
for i in range(30):
  obj = trainingStepBiasOnly(modelBiasOnly8, ratingsTrain, optimizer8)

In [ ]:
predictions = [modelBiasOnly8.predict(userIDs[u],itemIDs[i]).numpy() for u,i,_ in ratingsValid]
validMSE = MSE(predictions, labels)

In [ ]:
answers['Q8'] = (lamb8, validMSE)

In [ ]:
assertFloat(answers['Q8'][0])
assertFloat(answers['Q8'][1])

In [ ]:
predictions = open("drive/MyDrive/CSE 258/predictions_Rating.csv", 'w')
for l in open("drive/MyDrive/CSE 258/pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)

predictions.close()

In [ ]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()